In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import joblib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [15]:
df=pd.read_csv('preprocessed_dataset.csv')
df.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [16]:
df.columns

Index(['# mean_0_a', 'mean_1_a', 'mean_2_a', 'mean_3_a', 'mean_4_a',
       'mean_d_0_a', 'mean_d_1_a', 'mean_d_2_a', 'mean_d_3_a', 'mean_d_4_a',
       ...
       'fft_741_b', 'fft_742_b', 'fft_743_b', 'fft_744_b', 'fft_745_b',
       'fft_746_b', 'fft_747_b', 'fft_748_b', 'fft_749_b', 'label'],
      dtype='object', length=2549)

In [17]:

encode = ({'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': 2} )
df_encoded = df.replace(encode)

C:\Users\Praveen\AppData\Local\Temp\ipykernel_12848\3020761041.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df.replace(encode)


In [18]:
correlation_matrix = df_encoded.corr()
target_correlation = correlation_matrix['label'].abs()
sorted_correlation = target_correlation.sort_values(ascending=False)
top_75_features = sorted_correlation.iloc[1:76].index



In [19]:
# Step 2: Create X and y
X = df_encoded[top_75_features]  # Selecting the top 75 features
y = df_encoded['label'] 
y = to_categorical(y)         # Target column
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.reshape(X_train, (X_train.shape[0],1,X.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],1,X.shape[1]))
print(f"X_train  shape: {X_train.shape}")
print(f"X_test  shape: {X_test.shape}")
print(y_train.shape)  # Should be (num_samples, num_classes)
print(y_test.shape)


X_train  shape: (1705, 1, 75)
X_test  shape: (427, 1, 75)
(1705, 3)
(427, 3)


In [20]:
df_encoded[top_75_features]

,min_2_a,min_2_b,min_q_7_a,min_q_17_a,min_q_7_b,min_q_17_b,min_q_2_a,min_q_2_b,min_q_12_a,min_q_12_b,...,fft_409_a,fft_1_b,fft_4_b,fft_4_a,fft_1_a,fft_409_b,fft_406_b,fft_360_a,fft_315_a,fft_690_a
0,-727.000,-727.000,-676.000,-727.00,-727.00,-658.000,-677.00,-702.00,-698.00,-700.00,...,445.0,438.00,438.00,398.00,398.00,508.00,508.00,-508.0,-359.0,-462.0
1,-0.705,12.700,-0.705,12.00,12.70,13.500,11.10,16.00,16.30,17.70,...,-4.2,-4.00,-4.00,-4.30,-4.30,-5.64,-5.64,191.0,140.0,76.0
2,-844.000,-814.000,-844.000,-822.00,-803.00,-743.000,-707.00,-814.00,-826.00,-610.00,...,282.0,361.00,361.00,204.00,204.00,131.00,131.00,-57.2,-582.0,-125.0
3,-819.000,-820.000,-813.000,-712.00,-718.00,-552.000,-671.00,-820.00,-819.00,-584.00,...,-28.4,3.89,3.89,6.25,6.25,3.32,3.32,217.0,188.0,10.0
4,-9.640,-36.000,-5.480,-9.64,-12.30,7.600,12.20,19.40,20.10,-36.00,...,-55.6,-27.40,-27.40,-2.37,-2.37,56.40,56.40,350.0,203.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,7.940,-0.538,9.940,8.16,8.89,-0.538,7.94,11.60,11.20,5.95,...,27.3,3.37,3.37,12.00,12.00,-4.19,-4.19,142.0,112.0,187.0
2128,-756.000,-739.000,-756.000,-674.00,-670.00,-739.000,-639.00,-700.00,-689.00,-718.00,...,34.6,202.00,202.00,261.00,261.00,135.00,135.00,-29.9,-85.2,101.0
2129,-681.000,-650.000,-681.000,-651.00,-650.00,-599.000,-653.00,-580.00,-578.00,-574.00,...,382.0,312.00,312.00,302.00,302.00,314.00,314.00,-489.0,-387.0,-284.0
2130,-924.000,-933.000,-759.000,-924.00,-926.00,-933.000,-754.00,-835.00,-833.00,-813.00,...,-87.5,2.48,2.48,25.60,25.60,-62.10,-62.10,384.0,119.0,327.0


In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()


In [22]:
def get_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(y_train.shape[1], activation='softmax') 
    ])
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
model = get_cnn_model(input_shape)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=0)
model.save("../../Models/EEG/cnnmodel.h5")
score, acc = model.evaluate(X_test, y_test)
print(f"Accuracy : {acc}")

pred = model.predict(X_test)
predict_classes = np.argmax(pred, axis=1)
expected_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(expected_classes, predict_classes)
print("Confusion Matrix Accuracy: \n", conf_matrix)


c:\Users\Praveen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9141 - loss: 0.2325 
Accuracy : 0.9156908392906189
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Confusion Matrix Accuracy: 
 [[129  19   0]
 [  1 125  10]
 [  0   6 137]]


In [23]:
import random
import numpy as np

random_index = random.randint(0, X_test.shape[0] - 1)
random_sample = X_test[random_index].reshape(1, 1, X_test.shape[2])

predicted_probabilities = model.predict(random_sample)
predicted_class = np.argmax(predicted_probabilities, axis=1)[0]  # Convert prediction to class label

# Get the original class for the selected record
original_class = np.argmax(y_test[random_index])

# Print predicted and original results
print(f"Random Record Index: {random_index}")
print(f"Predicted Class: {predicted_class}")
print(f"Original Class: {original_class}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Random Record Index: 216
Predicted Class: 0
Original Class: 0


In [24]:
def get_LSTM_model():
    model = Sequential([
        LSTM(64, input_shape=(1,75),activation="relu",return_sequences=True),
        Dropout(0.2),
        LSTM(32,activation="relu",return_sequences=True),
        Dropout(0.2),
        LSTM(16,activation="relu"),
        Dropout(0.2),
        Dense(3, activation='softmax')
    ])
    
    return model

model = get_LSTM_model()


model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])


history = model.fit(X_train, y_train, epochs = 20, validation_data= (X_test, y_test), verbose = 0)
model.save("../../Models/EEG/lstmmodel.h5")
score, acc = model.evaluate(X_test, y_test)

print(f"Accuracy after Fine tuning Model {acc}")


pred = model.predict(X_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)

conf_matrix = confusion_matrix(expected_classes,predict_classes)
print("Confusion Matrix Accuracy: \n", conf_matrix)

c:\Users\Praveen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9130 - loss: 0.2237 
Accuracy after Fine tuning Model 0.908665120601654
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
Confusion Matrix Accuracy: 
 [[127  21   0]
 [  0 122  14]
 [  0   4 139]]


In [25]:
from sklearn.svm import SVC


svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train.reshape(X_train.shape[0], -1), np.argmax(y_train, axis=1))
joblib.dump(svm_model,"../../Models/EEG/svmmodel.joblib")
y_pred_svm = svm_model.predict(X_test.reshape(X_test.shape[0], -1))

accuracy_svm = accuracy_score(np.argmax(y_test, axis=1), y_pred_svm)
conf_matrix_svm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_svm)

print(f"Accuracy of SVM: {accuracy_svm}")
print("Confusion Matrix of SVM:\n", conf_matrix_svm)


Accuracy of SVM: 0.9086651053864169
Confusion Matrix of SVM:
 [[133  15   0]
 [  5 118  13]
 [  1   5 137]]


In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train.reshape(X_train.shape[0], -1), np.argmax(y_train, axis=1))
joblib.dump(decision_tree_model,"../../Models/EEG/dtmodel.joblib")
y_pred_dt = decision_tree_model.predict(X_test.reshape(X_test.shape[0], -1))

accuracy_dt = accuracy_score(np.argmax(y_test, axis=1), y_pred_dt)
conf_matrix_dt = confusion_matrix(np.argmax(y_test, axis=1), y_pred_dt)

print(f"Accuracy of Decision Tree: {accuracy_dt}")
print("Confusion Matrix of Decision Tree:\n", conf_matrix_dt)


Accuracy of Decision Tree: 0.9039812646370023
Confusion Matrix of Decision Tree:
 [[139   9   0]
 [  6 120  10]
 [  0  16 127]]


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train.reshape(X_train.shape[0], -1), np.argmax(y_train, axis=1))

y_pred_rf = random_forest_model.predict(X_test.reshape(X_test.shape[0], -1))

accuracy_rf = accuracy_score(np.argmax(y_test, axis=1), y_pred_rf)
conf_matrix_rf = confusion_matrix(np.argmax(y_test, axis=1), y_pred_rf)

print(f"Accuracy of Random Forest: {accuracy_rf}")
print("Confusion Matrix of Random Forest:\n", conf_matrix_rf)


Accuracy of Random Forest: 0.9508196721311475
Confusion Matrix of Random Forest:
 [[143   5   0]
 [  1 123  12]
 [  0   3 140]]


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix

def add_noise(data, noise_factor=0.2):  
    noisy_data = data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    return np.clip(noisy_data, 0., 1.)

def get_denoising_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    
    x = Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)  
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = MaxPooling1D(2, padding='same')(x)

    x = Conv1D(128, 3, activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = UpSampling1D(2)(x)
    x = Conv1D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling1D(2)(x)
    decoded = Conv1D(input_shape[-1], 3, activation='sigmoid', padding='same')(x)
    autoencoder = Model(input_layer, decoded)
    return autoencoder

def get_classifier_model(input_shape):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),  
        BatchNormalization(),  
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),  
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  
    ])
    return model

X_train_noisy = add_noise(X_train)
X_test_noisy = add_noise(X_test)

input_shape = (X_train.shape[1], X_train.shape[2])

autoencoder = get_denoising_autoencoder(input_shape)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

autoencoder.fit(X_train_noisy, X_train, epochs=50, batch_size=64, validation_data=(X_test_noisy, X_test), 
                callbacks=[early_stopping], verbose=1)

encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[4].output)  

X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

classifier = get_classifier_model(X_train_encoded.shape[1:])
classifier.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_train_encoded, y_train, epochs=50, batch_size=64, validation_data=(X_test_encoded, y_test), 
               callbacks=[early_stopping], verbose=1)

score, acc = classifier.evaluate(X_test_encoded, y_test)
print(f"Classifier Accuracy: {acc}")

y_pred = classifier.predict(X_test_encoded)
predict_classes = np.argmax(y_pred, axis=1)
expected_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(expected_classes, predict_classes)
print("Confusion Matrix:\n", conf_matrix)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 1.1107 - val_loss: 1.1333
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7836 - val_loss: 1.0048
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7292 - val_loss: 0.9064
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7320 - val_loss: 0.8483
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7305 - val_loss: 0.8090
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6986 - val_loss: 0.7859
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6458 - val_loss: 0.7644
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6768 - val_loss: 0.7488
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6750 - val_loss: 0.7361
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6676 - val_loss: 0.7265
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6985 - val_loss: 0.7213
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6873 - val_l

c:\Users\Praveen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4283 - loss: 1.6197 - val_accuracy: 0.7494 - val_loss: 0.6836
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6114 - loss: 0.9946 - val_accuracy: 0.7963 - val_loss: 0.5403
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7109 - loss: 0.7761 - val_accuracy: 0.8337 - val_loss: 0.4618
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7654 - loss: 0.6284 - val_accuracy: 0.8407 - val_loss: 0.4225
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7691 - loss: 0.5796 - val_accuracy: 0.8501 - val_loss: 0.3724
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7841 - loss: 0.5847 - val_accuracy: 0.8478 - val_loss: 0.3530
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8024 - loss: 0.5174 - val_accuracy: 0.8478 - val_loss: 0.3356
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8313 - loss: 0.4561 - val_accuracy: 0.8501 - val_loss: 0.3291
Ep

In [29]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LeakyReLU, Add, Input, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Model


def get_complex_classifier_model(input_shape):
    inputs = Input(shape=input_shape)

    x = Flatten()(inputs)
    x = Dense(256, kernel_regularizer=l2(0.001))(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x1 = Dense(128, kernel_regularizer=l2(0.001))(x)
    x1 = LeakyReLU(alpha=0.1)(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.5)(x1)

    x2 = Dense(128, kernel_regularizer=l2(0.001))(x1)
    x2 = LeakyReLU(alpha=0.1)(x2)
    x2 = BatchNormalization()(x2)

    x = Add()([x1, x2])
    x = Dropout(0.5)(x)

    x = Dense(64, kernel_regularizer=l2(0.001))(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(3, activation='softmax')(x)  
    model = Model(inputs, outputs)
    return model

complex_classifier = get_complex_classifier_model(X_train_encoded.shape[1:])
complex_classifier.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

complex_classifier.fit(X_train_encoded, y_train, epochs=50, batch_size=64, validation_data=(X_test_encoded, y_test), 
                       callbacks=[early_stopping], verbose=1)

score, acc = complex_classifier.evaluate(X_test_encoded, y_test)
print(f"Classifier Accuracy: {acc}")

y_pred = complex_classifier.predict(X_test_encoded)
predict_classes = np.argmax(y_pred, axis=1)
expected_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(expected_classes, predict_classes)
print("Confusion Matrix:\n", conf_matrix)


Epoch 1/50


c:\Users\Praveen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5033 - loss: 1.8109 - val_accuracy: 0.8431 - val_loss: 1.2237
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7685 - loss: 1.1819 - val_accuracy: 0.8478 - val_loss: 1.0163
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7974 - loss: 1.1338 - val_accuracy: 0.8548 - val_loss: 0.9411
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8211 - loss: 1.0390 - val_accuracy: 0.8665 - val_loss: 0.8895
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8422 - loss: 0.9568 - val_accuracy: 0.8642 - val_loss: 0.8463
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8465 - loss: 0.9966 - val_accuracy: 0.8782 - val_loss: 0.8350
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8476 - loss: 0.9250 - val_accuracy: 0.8806 - val_loss: 0.8118
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8500 - loss: 0.9227 - val_accuracy: 0.8923 - val_loss: 0.7954
Ep